--- Day 7: No Space Left On Device ---
You can hear birds chirping and raindrops hitting leaves as the expedition proceeds. Occasionally, you can even hear much louder sounds in the distance; how big do the animals get out here, anyway?

The device the Elves gave you has problems with more than just its communication system. You try to run a system update:

```
$ system-update --please --pretty-please-with-sugar-on-top
Error: No space left on device
```

Perhaps you can delete some files to make space for the update?

You browse around the filesystem to assess the situation and save the resulting terminal output (your puzzle input). For example:

```
$ cd /
$ ls
dir a
14848514 b.txt
8504156 c.dat
dir d
$ cd a
$ ls
dir e
29116 f
2557 g
62596 h.lst
$ cd e
$ ls
584 i
$ cd ..
$ cd ..
$ cd d
$ ls
4060174 j
8033020 d.log
5626152 d.ext
7214296 k
```

The filesystem consists of a tree of files (plain data) and directories (which can contain other directories or files). The outermost directory is called /. You can navigate around the filesystem, moving into or out of directories and listing the contents of the directory you're currently in.

Within the terminal output, lines that begin with $ are commands you executed, very much like some modern computers:
  - cd means change directory. This changes which directory is the current directory, but the specific result depends on the argument:
    - cd x moves in one level: it looks in the current directory for the directory named x and makes it the current directory.
    - cd .. moves out one level: it finds the directory that contains the current directory, then makes that directory the current directory.
    - cd / switches the current directory to the outermost directory, /.
  - ls means list. It prints out all of the files and directories immediately contained by the current directory:
    - 123 abc means that the current directory contains a file named abc with size 123.
    - dir xyz means that the current directory contains a directory named xyz.

Given the commands and output in the example above, you can determine that the filesystem looks visually like this:

```
- / (dir)
  - a (dir)
    - e (dir)
      - i (file, size=584)
    - f (file, size=29116)
    - g (file, size=2557)
    - h.lst (file, size=62596)
  - b.txt (file, size=14848514)
  - c.dat (file, size=8504156)
  - d (dir)
    - j (file, size=4060174)
    - d.log (file, size=8033020)
    - d.ext (file, size=5626152)
    - k (file, size=7214296)
```

Here, there are four directories: / (the outermost directory), a and d (which are in /), and e (which is in a). These directories also contain files of various sizes.

Since the disk is full, your first step should probably be to find directories that are good candidates for deletion. To do this, you need to determine the total size of each directory. The total size of a directory is the sum of the sizes of the files it contains, directly or indirectly. (Directories themselves do not count as having any intrinsic size.)

The total sizes of the directories above can be found as follows:
  - The total size of directory e is 584 because it contains a single file i of size 584 and no other directories.
  - The directory a has total size 94853 because it contains files f (size 29116), g (size 2557), and h.lst (size 62596), plus file i indirectly (a contains e which contains i).
  - Directory d has total size 24933642.
  - As the outermost directory, / contains every file. Its total size is 48381165, the sum of the size of every file.

To begin, find all of the directories with a total size of at most 100000, then calculate the sum of their total sizes. In the example above, these directories are a and e; the sum of their total sizes is 95437 (94853 + 584). (As in this example, this process can count files more than once!)

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [1]:
import json

In [3]:
# Get OR skip next line in input file; terminates at file end
def next_line(file, skip = False):
    line = file.readline().strip().split()
    return file if skip else line if line != [] else ["$", "cd", ".."]
 

In [4]:
def get_size(dir):
    return sum(get_size(sub_dir) if type(sub_dir) == dict else sub_dir for sub_dir in dir.values())

In [5]:
# Store home directory into dictionary
def add_directory(file, size):
    dir = {}
    line = next_line(file)
 
    while line[0] != "$":
        dir[line[1]] = {} if line[0] == "dir" else int(line[0])
        line = next_line(file)
 
    while line[2] != "..":
        if line[1] != "ls":
            dir[line[2]] = add_directory(next_line(file, True), size)
 
            # Part 1
            dir_size = get_size(dir[line[2]])
            size[0] += dir_size if dir_size <= 100000 else 0
 
        line = next_line(file)
 
    return dir

In [6]:
# Find smallest directory (size AND name) with enough space to create minimum unused space if deleted
def find_optimal_directory(dir, size, optimal_directory):
    for name, sub_dir in dir.items():
        if type(sub_dir) == dict:
            optimal_directory = find_optimal_directory(sub_dir, size, optimal_directory)
            sub_dir_size = get_size(sub_dir)
 
            if sub_dir_size > size and sub_dir_size < optimal_directory[0]:
                optimal_directory = [sub_dir_size, name]
 
    return optimal_directory

In [12]:
if __name__ == "__main__":
    # Get data from today's AoC input file + Part 1
    with open("d7.txt") as file:
        small_dirs = [0] # Total size of dirs <= 100000
        home = {next_line(file)[2]: add_directory(next_line(file, True), small_dirs)}
        print(json.dumps(home, indent = 16)) # Print home directory (pretty though)
 
    # Part 2
    TOTAL_DISK_SPACE = 70000000
    MINIMUM_UNUSED_SPACE = 30000000
    home_size = get_size(home)
 
    space_needed = MINIMUM_UNUSED_SPACE - TOTAL_DISK_SPACE + home_size
    optimal_directory = find_optimal_directory(home, space_needed, [home_size, None]) # Size, name
 

{
                "/": {
                                "ccjp": {
                                                "dlz": 159990,
                                                "mbtsvblj": {
                                                                "frqsf.nsv": 34806,
                                                                "ppq": {
                                                                                "dhzp": 266252
                                                                },
                                                                "ptht": {
                                                                                "jbnj": {
                                                                                                "clscr": {
                                                                                                                "dhzp": 249531
                                                                                           

In [13]:

    # Display
    print("Part 1: ", small_dirs[0], "\nPart 2: ", optimal_directory[0])
 

Part 1:  1513699 
Part 2:  7991939
